In [3]:

from tqdm import tqdm
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from pathlib import Path

In [14]:
data_list = []

for path in Path("sloppy_ocr_manual_text").glob("*.txt"):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        text = text.replace("\n", " ") #just in case some nonsense occurred in the ocr
    
    data_list.append((path.name, text))

In [15]:
df = pd.DataFrame(data_list, columns=['file_name', 'text'])